<a href="https://colab.research.google.com/github/jenmusdat/Project3_BeerQuality/blob/main/Analysis/training_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Start a SparkSession
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [4]:
MAX_MEMORY = "5g"

spark = SparkSession \
    .builder \
    .appName("word2vec") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.driver.maxResultSize", MAX_MEMORY)\
    .getOrCreate()

In [5]:
# Import needed libs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string


In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), header=True)


In [7]:
# remove alpha characters from data set on review numerics
from pyspark.sql.functions import regexp_extract, col
pattern = r'^\d+.\d+$'
df = df.filter(regexp_extract(col('look'), pattern, 0) != '')

In [8]:
df = df.filter(regexp_extract(col('smell'), pattern, 0) != '')
df = df.filter(regexp_extract(col('taste'), pattern, 0) != '')
df = df.filter(regexp_extract(col('overall'), pattern, 0) != '')
df = df.filter(regexp_extract(col('score'), pattern, 0) != '')
df = df.filter(regexp_extract(col('feel'), pattern, 0) != '')

In [9]:
# Next 6 lines casts to floats all the review ratings
df = df.withColumn('look_numeric', df['look'].cast('float'))

In [10]:
df = df.withColumn('smell_numeric', df['smell'].cast('float'))

In [11]:
df = df.withColumn('taste_numeric', df['taste'].cast('float'))

In [12]:
df = df.withColumn('feel_numeric', df['feel'].cast('float'))

In [13]:
df = df.withColumn('overall_numeric', df['overall'].cast('float'))

In [14]:
df = df.withColumn('score_numeric', df['score'].cast('float'))

In [15]:
# Confirm data types
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string, look_numeric: float, smell_numeric: float, taste_numeric: float, feel_numeric: float, overall_numeric: float, score_numeric: float]>

In [16]:
# functions for manipulating review text into a format we can tokenize
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'
  

In [17]:
# define the udfs for the data
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [18]:
# clean the beer name by removing spaces (will be combined with text)
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
# beer_df.show()

In [19]:
# clean the style name by removing spaces (will be combined with text)
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
# style_df.show()

In [20]:
# clean the text by removing punctuation
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-

In [21]:
# Combine beer_name, style, and review text into a Final_text column
from pyspark.sql.functions import concat_ws

combined_df = text_df.withColumn('Final_text', concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text))
combined_df.select('text', 'Final_text').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [26]:
# Setup and execute Tokenizer and Stop words pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)


In [ ]:
# final_df.show()

In [27]:
final_df = final_df.select('score_numeric','features')

In [28]:
final_df.show()

+-------------+--------------------+
|score_numeric|            features|
+-------------+--------------------+
|         4.03|[motorbreathimper...|
|          3.7|[paybackpilsner, ...|
|         4.08|[paybackpilsner, ...|
|         3.43|[paybackpilsner, ...|
|         3.58|[paybackpilsner, ...|
|         3.52|[paybackpilsner, ...|
|         3.36|[paybackpilsner, ...|
|         3.88|[paybackpilsner, ...|
|         3.61|[paybackpilsner, ...|
|         3.53|[paybackpilsner, ...|
|         3.38|[hellhammer, ryeb...|
|         4.24|[hellhammer, ryeb...|
|          3.0|[vanillaporter, a...|
|         3.85|[vanillaporter, a...|
|          3.5|[vanillaporter, a...|
|         3.53|[vanillaporter, a...|
|         2.52|[vanillaporter, a...|
|          3.9|[vanillaporter, a...|
|         4.04|[vanillaporter, a...|
|         3.65|[vanillaporter, a...|
+-------------+--------------------+
only showing top 20 rows



In [29]:
# Read the model from disk to confirm
w2v_loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [30]:
# check to see if synonym is consistent with loaded model
w2v_loaded_model.findSynonymsArray("light", 5)

[('mild', 0.6426884531974792),
 ('faint', 0.6223998665809631),
 ('slight', 0.6191715002059937),
 ('mostly', 0.6169756650924683),
 ('medium', 0.6095581650733948)]

In [31]:
# Transform the mode
w2v_model_df = w2v_loaded_model.transform(final_df)

In [32]:
w2v_model_df.show()

+-------------+--------------------+--------------------+
|score_numeric|            features|               model|
+-------------+--------------------+--------------------+
|         4.03|[motorbreathimper...|[0.02950784611367...|
|          3.7|[paybackpilsner, ...|[-0.0872463285923...|
|         4.08|[paybackpilsner, ...|[0.01790586973612...|
|         3.43|[paybackpilsner, ...|[-0.1276244542612...|
|         3.58|[paybackpilsner, ...|[-0.0131208907440...|
|         3.52|[paybackpilsner, ...|[-0.0353247860187...|
|         3.36|[paybackpilsner, ...|[-0.0606238029503...|
|         3.88|[paybackpilsner, ...|[0.00897214795847...|
|         3.61|[paybackpilsner, ...|[0.02342551416502...|
|         3.53|[paybackpilsner, ...|[-0.0209101041158...|
|         3.38|[hellhammer, ryeb...|[0.01432307037573...|
|         4.24|[hellhammer, ryeb...|[-0.1081325657665...|
|          3.0|[vanillaporter, a...|[-0.1101182210924...|
|         3.85|[vanillaporter, a...|[-0.0197092709394...|
|          3.5

In [33]:
w2v_model_df = w2v_model_df.select('score_numeric', 'model')

In [34]:
# Prepare the hyperparameters
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
lin_model = LinearRegression(
   maxIter=5,
   regParam=0.3,
   featuresCol="model",
   labelCol="score_numeric"
)

In [35]:
w2v_model_df.cache()
w2v_model_df.show()

+-------------+--------------------+
|score_numeric|               model|
+-------------+--------------------+
|         4.03|[0.02950784611367...|
|          3.7|[-0.0872463285923...|
|         4.08|[0.01790586973612...|
|         3.43|[-0.1276244542612...|
|         3.58|[-0.0131208907440...|
|         3.52|[-0.0353247860187...|
|         3.36|[-0.0606238029503...|
|         3.88|[0.00897214795847...|
|         3.61|[0.02342551416502...|
|         3.53|[-0.0209101041158...|
|         3.38|[0.01432307037573...|
|         4.24|[-0.1081325657665...|
|          3.0|[-0.1101182210924...|
|         3.85|[-0.0197092709394...|
|          3.5|[-0.0049533498369...|
|         3.53|[0.02202844081446...|
|         2.52|[-0.0874559852428...|
|          3.9|[-0.0728758174786...|
|         4.04|[-0.1125914826324...|
|         3.65|[-0.1178625054068...|
+-------------+--------------------+
only showing top 20 rows



In [36]:
# split training and testing for the model (for temporary make the training set small as processing takes time)
training, testing = w2v_model_df.randomSplit([0.80, 0.20])

In [37]:
training.show()

+-------------+--------------------+
|score_numeric|               model|
+-------------+--------------------+
|          1.0|[-0.1452391079316...|
|          1.0|[-0.1097804032266...|
|          1.0|[-0.1004139792794...|
|          1.0|[-0.0677027806639...|
|          1.0|[-0.0668491764137...|
|          1.0|[-0.0632363053903...|
|          1.0|[-0.0606100791133...|
|          1.0|[-0.0447331815958...|
|          1.0|[-0.0392828791445...|
|          1.0|[-0.0207084439073...|
|          1.0|[-0.0202475327532...|
|          1.0|[-0.0109136197715...|
|          1.0|[0.00501279958656...|
|          1.0|[0.02593322018427...|
|          1.0|[0.03135539820466...|
|          1.0|[0.03472432556359...|
|          1.0|[0.03590444286415...|
|          1.0|[0.03657745135327...|
|          1.0|[0.04092206743856...|
|          1.0|[0.04913005432235...|
+-------------+--------------------+
only showing top 20 rows



In [38]:
training_pandas_df = training.toPandas()

In [39]:
training_pandas_df.isnull().sum()

score_numeric    0
model            0
dtype: int64

In [40]:
training_pandas_df = training_pandas_df.dropna()

In [41]:
training_pandas_df.head()

,score_numeric,model
0,1.0,"[-0.14523910793165368, 0.12042828451505759, -0..."
1,1.0,"[-0.10978040322661399, 0.04642222771154982, -0..."
2,1.0,"[-0.10041397927949826, -0.0005390884975592296,..."
3,1.0,"[-0.06770278066396714, 0.01826591268181801, -0..."
4,1.0,"[-0.06684917641373782, 0.017635300278645728, -..."


In [42]:
training_pandas_df.dtypes

score_numeric    float32
model             object
dtype: object

In [43]:
from sklearn.linear_model import LinearRegression
pandas_lin_model = LinearRegression()

In [44]:
lin_score_train_model = pandas_lin_model.fit(training_pandas_df['model'].to_list(),training_pandas_df['score_numeric'])

In [45]:
lin_score_train_model.score(training_pandas_df['model'].to_list(), training_pandas_df['score_numeric'])

0.4535757942973402

In [46]:
lin_score_train_model.coef_

array([ 0.26890903,  0.97758512,  0.9453348 , -0.37180642,  0.41379717,
        1.40873703,  1.04119982,  1.21264283,  0.01570631, -0.20795607,
        0.15449696, -0.36062022,  0.94182257,  0.44139991,  0.50686849,
        0.60096441, -0.78010429,  0.45978745, -0.07100689, -0.64681504,
       -0.21051633, -0.68548655,  0.02901754, -0.22055218,  0.31393899,
       -0.31268129, -0.30313675, -0.75313916, -0.7542196 ,  1.0451003 ,
        0.79059547, -0.37880137,  0.49814205, -0.04382529,  0.29582802,
       -0.86629356, -0.29978127,  0.36787168,  0.19895674, -0.09194922,
        0.10716205, -0.92708217,  0.60523246, -0.82026928, -0.02110568,
       -0.31940939, -0.70202255, -0.99005725, -1.7051153 , -0.24391419,
       -0.03935805,  0.98911055,  0.57247967,  0.38740594, -0.91632911,
        0.47994815, -0.26637865, -0.03296762, -0.70038526,  0.59481876,
        0.44944602,  0.2791702 , -0.57581321,  1.32920634,  0.82427327,
       -0.55006714, -0.17627139, -0.72228962, -0.16455984, -0.08

In [47]:
# Fit the score review to a model (takes time)
lin_score_train_model = lin_model.fit(training)

In [48]:
# Save the model for later use
lin_score_train_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/lin_score_train.md")

In [49]:
# Read the model
loaded_lin_score_train_model = LinearRegressionModel.load("/content/gdrive/MyDrive/Project3_BeerQuality/lin_score_train.md")

In [50]:
# Transform the training data to peek at the predictions
lin_score_df = loaded_lin_score_train_model.transform(training)

In [51]:
lin_score_df.show()

+-------------+--------------------+------------------+
|score_numeric|               model|        prediction|
+-------------+--------------------+------------------+
|          1.0|[-0.1452391079316...|3.3509315942593845|
|          1.0|[-0.1097804032266...| 2.993971131857937|
|          1.0|[-0.1004139792794...|  2.83239251145132|
|          1.0|[-0.0677027806639...|4.0640070514729665|
|          1.0|[-0.0668491764137...| 3.947150655092111|
|          1.0|[-0.0632363053903...| 3.480358688961557|
|          1.0|[-0.0606100791133...| 2.921463532207384|
|          1.0|[-0.0447331815958...|3.4009977154668176|
|          1.0|[-0.0392828791445...| 2.533553596866348|
|          1.0|[-0.0207084439073...| 3.919159629244354|
|          1.0|[-0.0202475327532...| 2.627647277517573|
|          1.0|[-0.0109136197715...|3.7871159757803703|
|          1.0|[0.00501279958656...|3.2359822053820957|
|          1.0|[0.02593322018427...| 3.542579010127546|
|          1.0|[0.03135539820466...| 3.646984292

In [52]:
# show a sample of the predictions
lin_score_df.select('score_numeric', 'prediction').show(truncate=False)

+-------------+------------------+
|score_numeric|prediction        |
+-------------+------------------+
|1.0          |3.3509315942593845|
|1.0          |2.993971131857937 |
|1.0          |2.83239251145132  |
|1.0          |4.0640070514729665|
|1.0          |3.947150655092111 |
|1.0          |3.480358688961557 |
|1.0          |2.921463532207384 |
|1.0          |3.4009977154668176|
|1.0          |2.533553596866348 |
|1.0          |3.919159629244354 |
|1.0          |2.627647277517573 |
|1.0          |3.7871159757803703|
|1.0          |3.2359822053820957|
|1.0          |3.542579010127546 |
|1.0          |3.646984292891589 |
|1.0          |3.2487545849670387|
|1.0          |3.7216319112559972|
|1.0          |3.5534103575494638|
|1.0          |3.6132749074286847|
|1.0          |3.6186949193848745|
+-------------+------------------+
only showing top 20 rows



In [53]:
gr = lin_score_df.groupBy("score_numeric").count()
gr.show()

+-------------+-----+
|score_numeric|count|
+-------------+-----+
|         3.32| 3418|
|         4.96|  901|
|         1.63|  205|
|         3.36| 3921|
|         3.22| 2318|
|         4.04| 9542|
|         1.13|   53|
|         3.81| 7322|
|         4.03|19391|
|         4.63| 3153|
|         4.91|  879|
|         1.47|  103|
|         1.76|  274|
|         4.95| 1056|
|         1.08|   81|
|         2.54|  599|
|         2.96| 1786|
|         4.43| 6508|
|         4.85| 3232|
|          5.0|14464|
+-------------+-----+
only showing top 20 rows



In [54]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!
